## Sentiment Analysis

In [1]:
!pip install tkseem
!pip install tnkeeh

In [2]:
!wget https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/sentiment_analysis/sentiment/data.txt
!wget https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/sentiment_analysis/sentiment/labels.txt

--2020-08-01 18:09:58--  https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/sentiment_analysis/sentiment/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122098 (119K) [text/plain]
Saving to: ‘data.txt.1’

data.txt.1          100%[===================>] 119.24K   380KB/s    in 0.3s    

2020-08-01 18:10:00 (380 KB/s) - ‘data.txt.1’ saved [122098/122098]

--2020-08-01 18:10:00--  https://raw.githubusercontent.com/ARBML/tkseem/master/tasks/sentiment_analysis/sentiment/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3999 (3.9K) [text/plain]
Saving to: ‘labels.txt.1’

labels.txt.1        

### Imports

In [1]:
import numpy as np
import tkseem as tk
import tnkeeh as tn
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional

### Process data

In [2]:
tn.clean_data(file_path = 'sentiment/data.txt', save_path = 'sentiment/cleaned_data.txt', remove_diacritics=True, 
      execluded_chars=['!', '.', '?'])
tn.split_classification_data('sentiment/cleaned_data.txt', 'sentiment/labels.txt')
train_data, test_data, train_lbls, test_lbls = tn.read_data(mode = 1)

Remove diacritics
Remove Tatweel
Saving to sentiment/cleaned_data.txt
Split data
Save to data
Read data  ['test_data.txt', 'test_lbls.txt', 'train_data.txt', 'train_lbls.txt']


In [3]:
max_length = max(len(data) for data in train_data)

### Tokenize

In [4]:
tokenizer = tk.SentencePieceTokenizer()
tokenizer.train('data/train_data.txt')

Training SentencePiece ...


### Tokenize data

In [5]:
def preprocess(tokenizer, data, labels):
    X = tokenizer.encode_sentences(data)
    y = np.array([int(lbl) for lbl in labels])
    return X, y

In [6]:
# process training data
X_train, y_train = preprocess(tokenizer, train_data, train_lbls)

# process test data
X_test, y_test = preprocess(tokenizer, test_data, test_lbls)

### Model

In [7]:
model = Sequential()
model.add(Embedding(tokenizer.vocab_size, 32))
model.add(Bidirectional(GRU(units = 32)))
model.add(Dense(32, activation = 'tanh'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Train

In [8]:
history = model.fit(X_train, y_train, epochs = 12, validation_split = 0.1,  batch_size= 128, shuffle = True)

Epoch 1/12
6/6 [==============================] - 1s 143ms/step - loss: 0.6928 - accuracy: 0.5111 - val_loss: 0.7008 - val_accuracy: 0.3625
Epoch 2/12
6/6 [==============================] - 0s 60ms/step - loss: 0.6867 - accuracy: 0.5611 - val_loss: 0.6947 - val_accuracy: 0.4250
Epoch 3/12
6/6 [==============================] - 0s 62ms/step - loss: 0.6788 - accuracy: 0.7153 - val_loss: 0.6863 - val_accuracy: 0.6375
Epoch 4/12
6/6 [==============================] - 0s 64ms/step - loss: 0.6631 - accuracy: 0.8361 - val_loss: 0.6758 - val_accuracy: 0.6625
Epoch 5/12
6/6 [==============================] - 0s 60ms/step - loss: 0.6355 - accuracy: 0.8944 - val_loss: 0.6683 - val_accuracy: 0.7000
Epoch 6/12
6/6 [==============================] - 0s 61ms/step - loss: 0.5879 - accuracy: 0.9153 - val_loss: 0.6526 - val_accuracy: 0.7000
Epoch 7/12
6/6 [==============================] - 0s 59ms/step - loss: 0.5109 - accuracy: 0.9250 - val_loss: 0.6102 - val_accuracy: 0.7250
Epoch 8/12
6/6 [==========

### Test

In [9]:
def classify(sentence):
  sequence = tokenizer.encode_sentences([sentence], out_length = max_length)[0]
  pred = model.predict(sequence)[0][0]
  print(pred)

In [10]:
classify("سيئة جدا جدا")
classify("رائعة جدا")

0.08256233
0.88681066
